In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

vehicles_dataset = pd.read_csv('/home/jovyan/work/datasets/vehicles_cleaned.csv')
vehicles_dataset

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,state
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc
...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,maxima s sedan 4d,gas,32226.0,clean,other,sedan,wy
426876,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,gas,12029.0,clean,other,sedan,wy
426877,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,diesel,4174.0,clean,other,hatchback,wy
426878,wyoming,28990,2018.0,lexus,es 350 sedan 4d,gas,30112.0,clean,other,sedan,wy


In [2]:
y = vehicles_dataset['price']
X = vehicles_dataset.copy()
X.drop(columns=['price'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.6,
    random_state=42
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test,
    y_test,
    train_size=0.5,
    random_state=42
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((256128, 10), (256128,), (85376, 10), (85376,), (85376, 10), (85376,))

In [3]:
X_train.reset_index(inplace=True, drop=True)
cols_with_null = [col for col in X_train.columns
                 if X_train[col].isnull().any()]
cols_with_null, X_train.columns

(['year',
  'manufacturer',
  'model',
  'fuel',
  'odometer',
  'title_status',
  'transmission',
  'type'],
 Index(['region', 'year', 'manufacturer', 'model', 'fuel', 'odometer',
        'title_status', 'transmission', 'type', 'state'],
       dtype='object'))

In [4]:
for col_with_null in cols_with_null:
    print(X_train[col_with_null].value_counts(dropna=False))

year
2018.0    21799
2017.0    21747
2015.0    19016
2013.0    18486
2016.0    18367
          ...  
1920.0        1
1913.0        1
1918.0        1
1901.0        1
1943.0        1
Name: count, Length: 112, dtype: int64
manufacturer
ford               42587
chevrolet          33119
toyota             20426
honda              12784
jeep               11603
nissan             11405
ram                10902
NaN                10624
gmc                 9975
bmw                 8756
dodge               8255
mercedes-benz       7066
hyundai             6149
subaru              5702
volkswagen          5598
kia                 5097
lexus               4911
audi                4630
cadillac            4172
chrysler            3560
acura               3547
buick               3290
mazda               3281
infiniti            2911
lincoln             2509
volvo               2069
mitsubishi          1968
mini                1416
pontiac             1384
rover               1284
jaguar           

In [11]:
manufacturer_null_values_row_indexes = X_train[X_train['manufacturer'].isnull()].index.tolist()
X_train_imputed_ffill = X_train.copy()
X_train_imputed_bfill = X_train.copy()
X_train_imputed_mode = X_train.copy()

print(
    X_train_imputed_ffill['manufacturer'].isnull().any(),
    X_train_imputed_bfill['manufacturer'].isnull().any(),
    X_train_imputed_mode['manufacturer'].isnull().any()
)

X_train_imputed_ffill['manufacturer'].ffill(inplace=True)
X_train_imputed_bfill['manufacturer'].bfill(inplace=True)
X_train_imputed_mode['manufacturer'].fillna(
    value=X_train_imputed_mode['manufacturer'].mode(), 
    inplace=True
)

print(
    X_train_imputed_ffill['manufacturer'].isnull().any(),
    X_train_imputed_bfill['manufacturer'].isnull().any(),
    X_train_imputed_mode['manufacturer'].isnull().any()
)

for row_imputed in manufacturer_null_values_row_indexes:
    print(
        X_train_imputed_ffill['manufacturer'][row_imputed],
        '-----------',
        X_train_imputed_bfill['manufacturer'][row_imputed],
        '------------',
        X_train_imputed_mode['manufacturer'][row_imputed],
    )

True True True
False False True
infiniti ----------- jeep ------------ nan
infiniti ----------- jeep ------------ nan
jeep ----------- ford ------------ nan
hyundai ----------- ford ------------ nan
ford ----------- dodge ------------ nan
chevrolet ----------- lexus ------------ nan
ford ----------- chevrolet ------------ nan
ford ----------- chevrolet ------------ nan
honda ----------- chevrolet ------------ nan
buick ----------- chevrolet ------------ nan
nissan ----------- fiat ------------ nan
chevrolet ----------- honda ------------ nan
toyota ----------- dodge ------------ nan
honda ----------- hyundai ------------ nan
bmw ----------- ford ------------ nan
ford ----------- chevrolet ------------ nan
ram ----------- nissan ------------ nan
nissan ----------- buick ------------ nan
chevrolet ----------- gmc ------------ nan
honda ----------- ford ------------ nan
chevrolet ----------- toyota ------------ nan
honda ----------- honda ------------ nan
honda ----------- mazda ---------

In [6]:
X_train['manufacturer'].mode()

0    ford
Name: manufacturer, dtype: object